<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Objectives" data-toc-modified-id="Objectives-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Why-$k$-means-Isn't-Enough" data-toc-modified-id="Why-$k$-means-Isn't-Enough-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Why $k$-means Isn't Enough</a></span></li><li><span><a href="#Hierarchical-Agglomerative-Clustering" data-toc-modified-id="Hierarchical-Agglomerative-Clustering-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Hierarchical Agglomerative Clustering</a></span><ul class="toc-item"><li><span><a href="#How-Does-the-Algorithm-Work" data-toc-modified-id="How-Does-the-Algorithm-Work-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>How Does the Algorithm Work</a></span><ul class="toc-item"><li><span><a href="#Agglomerative-Example" data-toc-modified-id="Agglomerative-Example-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Agglomerative Example</a></span></li></ul></li><li><span><a href="#Visualizing-with-Dendrograms" data-toc-modified-id="Visualizing-with-Dendrograms-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Visualizing with Dendrograms</a></span><ul class="toc-item"><li><span><a href="#Advantages-to-Using-Dendrograms" data-toc-modified-id="Advantages-to-Using-Dendrograms-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Advantages to Using Dendrograms</a></span></li><li><span><a href="#ASIDE:-How-Well-Does-the-Dendrogram-Fit-the-Data?" data-toc-modified-id="ASIDE:-How-Well-Does-the-Dendrogram-Fit-the-Data?-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>ASIDE: How Well Does the Dendrogram Fit the Data?</a></span></li></ul></li></ul></li><li><span><a href="#Types-of-Hierarchical-Agglomerative-Clustering" data-toc-modified-id="Types-of-Hierarchical-Agglomerative-Clustering-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Types of Hierarchical Agglomerative Clustering</a></span></li><li><span><a href="#Seeing-it-in-action" data-toc-modified-id="Seeing-it-in-action-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Seeing it in action</a></span><ul class="toc-item"><li><span><a href="#Hierarchical-clustering-with-scipy" data-toc-modified-id="Hierarchical-clustering-with-scipy-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Hierarchical clustering with <code>scipy</code></a></span></li><li><span><a href="#Hierarchical-clustering-with-sklearn-on-Iris" data-toc-modified-id="Hierarchical-clustering-with-sklearn-on-Iris-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Hierarchical clustering with <code>sklearn</code> on Iris</a></span></li><li><span><a href="#Evaluate" data-toc-modified-id="Evaluate-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Evaluate</a></span><ul class="toc-item"><li><span><a href="#Evaluating-number-of-clusters-/-Cut-points" data-toc-modified-id="Evaluating-number-of-clusters-/-Cut-points-5.3.1"><span class="toc-item-num">5.3.1&nbsp;&nbsp;</span>Evaluating number of clusters / Cut points</a></span></li></ul></li></ul></li><li><span><a href="#Advantages-&amp;-Disadvantages-of-hierarchical-clustering" data-toc-modified-id="Advantages-&amp;-Disadvantages-of-hierarchical-clustering-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Advantages &amp; Disadvantages of hierarchical clustering</a></span><ul class="toc-item"><li><span><a href="#Advantages" data-toc-modified-id="Advantages-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Advantages</a></span></li><li><span><a href="#Disadvantages" data-toc-modified-id="Disadvantages-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Disadvantages</a></span><ul class="toc-item"><li><span><a href="#Further-reading" data-toc-modified-id="Further-reading-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>Further reading</a></span></li></ul></li></ul></li></ul></div>

In [ ]:
import sys, os

ex_path = os.path.abspath(os.pardir)
if ex_path not in sys.path:
    sys.path.append(ex_path)

from src.hier_example import *
from sklearn.datasets import make_blobs
from src.av_link_agglom_clust import centrAggClust as ac
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet
from scipy.spatial.distance import pdist
from sklearn.datasets import make_blobs, make_moons, load_iris
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import KernelDensity
from sklearn.metrics import silhouette_score

%matplotlib inline

# Objectives

- Describe the algorithm for agglomerative hierarchical clustering
- Compare and contrast hierarchical clustering with $k$-means clustering
- Implement hierarchical clustering with `scipy` and `sklearn`
- Build and interpret dendrograms

# Why $k$-means Isn't Enough

> Recall that $k$-means clustering assigns individual observations to a _pre-specified number of clusters_ according to the distance between the centroid and the observation.

One disadvantage of $k$-means clustering is that we have to specify the number of clusters beforehand.

Clustering problems are sometimes difficult to say when you got it "right" since technically, there isn't necessarily a right answer. It really depends on the _context_ of the data.

Scikit-Learn shows some examples of different clustering algorithms on different datasets in its [documentation](https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html): 
![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cluster_comparison_001.png)

# Hierarchical Agglomerative Clustering

> Hierarchical agglomerative clustering sets out to group the most similar two observations together from a bottom-up level. 

The type of hierarchical clustering we will learn today is **agglomerative**, or **bottom-up**. We don't have to specify the number of clusters beforehand.

## How Does the Algorithm Work

Let's start with some points that we want to cluster (ignore the colors for now):

<img src="images/pre_cluster_points.png" width=30%/>

> Picture what you would picture to be reasonable groups.

- In the algorithm, initially every observation is its own cluster.

- Then we group points that are _most similar (nearest)_ to each other as a cluster.

- As we get small clusters, we group already formed clusters with nearby points and other clusters

This ultimately creates one large cluster but with smaller clusters making up the larger clusters. Each "smaller" cluster's members are overall more like each other than with the larger clusters.

> We observe "clusters within clusters"

![](images/yo_dawg_clusters_with_clusters.jpeg)

##### Results from Algorithm

|BEFORE | AFTER |
|:-----:| :---: |
|<img src="images/pre_cluster_points.png" width=60%/> | <img src="images/clustergram.png" width=100%/> |

### Agglomerative Example

In [ ]:
plot_agglomerative_algorithm()

## Visualizing with Dendrograms

We can also visualize the clustering with a tree-like diagram called a **dendrogram**.

|Clustering | Dendrograms |
|:-----:| :---: |
|<img src="images/clustergram.png" width=100%/> | <img src="images/dendrogram.png" width=100%/> |

This allows us to view the clusterings obtained for each possible number of clusters, from 1 to n. It is up to our discretion as data scientists to decide how many clusters we want. 

As we move up the leaf of the dendrogram, the two most similar observations fuse together, and then the next most similar clusters fuse together etc. until everything fuses together into a big cluster.

### Advantages to Using Dendrograms

- Useful in visualizing HIGH dimensional separation
- Can choose just "n" clusters
    + cut the link of the highest tree for two clusters
    + clusters are more alike

### ASIDE: How Well Does the Dendrogram Fit the Data?

One way of computing this is by means of the **cophenetic correlation coefficient**, which, in a word, is a measure of "how faithfully the dendrogram preserves the pairwise distances between the \[datapoints\]" -- [Wikipedia](https://en.wikipedia.org/wiki/Cophenetic_correlation).

The cophenetic correlation coefficient $c$ is given by [ref](https://www.mathworks.com/help/stats/index.html?/access/helpdesk/help/toolbox/stats/cophenet.html=)

![c-coef](images/cophenet.png)

$x(i, j) = | Xi − Xj |$, the ordinary Euclidean distance between the $i$th and $j$th observations.<br>
$t(i, j)$ = the dendrogrammatic distance between the model points $Ti$ and $Tj$. This distance is the height of the node at which these two points are first joined together.<br>

Then, letting ${\bar {x}}$ be the average of the $x(i, j)$, and letting ${\bar {t}}$ be the average of the $t(i, j)$, the cophenetic correlation coefficient $c$ is given by[4]

This is complicated! [This site](https://people.revoledu.com/kardi/tutorial/Clustering/Online-Hierarchical-Clustering.html) is helpful on how to understand the cophenetic correlation, and, indeed, on hierarchical clustering generally.

# Types of Hierarchical Agglomerative Clustering 

The way that distance is measured between clusters is called the model's **linkage**.

- Single Linkage 
    -  Minimum pair-wise distance: for any two clusters, take one observation from each and determine their distance. Do this over and over, until you have identified the overall minimum pair-wise distance. 
- Complete Linkage
    -  Complete linkage may be defined as the furthest (or maximum) distance between two clusters. That is, all possible pairwise distances between elements (one from cluster A and one from B) are evaluated and the largest value is used as the distance between clusters A & B. This is sometimes called complete linkage and is also called furthest neighbor.
- Average Linkage
    - The distance between clusters is defined as the average distance between the data points in the clusters. 
- Ward Linkage
    -  Ward method finds the pair of clusters that leads to minimum increase in total within-cluster variance after merging at each step.

[This article](https://towardsdatascience.com/understanding-the-concept-of-hierarchical-clustering-technique-c6e8243758ec) describes the pros and cons of each approach.

# Seeing it in action

[This post here](https://www.analyticsvidhya.com/blog/2019/05/beginners-guide-hierarchical-clustering/) walks through cluster assignment _step_ by _step_.

In [ ]:
X, Y = make_blobs(n_samples=10, n_features=1,
                  centers=5, random_state=42)
ac(X, Y)

In [ ]:
X2 = [1, 2, 5, 7, 9]
Y2 = [2, 2, 8, 2, 2]

ac(X2, Y2)

Meanwhile, we can do it in _**scipy**_ and _**sklearn**_

## Hierarchical clustering with `scipy`

In [ ]:
# Let's generate some data and look at an example of
# hierarchical agglomerative clustering.
# Generate two clusters: a with 100 points, b with 50:

np.random.seed(1000)

a = np.random.multivariate_normal([10, 0], [[3, 1], [1, 4]], size=[100,])
b = np.random.multivariate_normal([0, 20], [[3, 1], [1, 4]], size=[50,])
X = np.concatenate((a, b))

print(X.shape)  # 150 samples with 2 dimensions
fig, ax = plt.subplots()
ax.scatter(X[:, 0], X[:, 1])
ax.set_title("Sample data for clustering demo");

In [ ]:
# construct dendrogram in scipy

Z = linkage(X, 'single')

In [ ]:
Z.shape

In [ ]:
Z[:5, :]

Z contains the dendrogram in coded form! Its third column has distances. https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html

`pdist()` calculates pairwise distances for an input array of points.

In [ ]:
X[:3, :]

In [ ]:
print(((X[0, 0] - X[1, 0])**2 + (X[0, 1] - X[1, 1])**2)**0.5)
print(((X[0, 0] - X[2, 0])**2 + (X[0, 1] - X[2, 1])**2)**0.5)
print(((X[0, 0] - X[3, 0])**2 + (X[0, 1] - X[3, 1])**2)**0.5)

In [ ]:
pdist(X)[:3]

In [ ]:
len(X)

In [ ]:
len(pdist(X))

In [ ]:
# For each point, there are 149 distances
# to other points.

150*149 // 2

In [ ]:
c, coph_dists = cophenet(Z, pdist(X))
c

In [ ]:
coph_dists[:5]

In [ ]:
# calculate and construct the dendrogram 
# calculate full dendrogram

fig, ax = plt.subplots(figsize=(25, 10))
ax.set_title('Hierarchical Clustering Dendrogram')
ax.set_xlabel('sample index')
ax.set_ylabel('distance')
dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=8.,  # font size for the x axis labels
);

In [ ]:
# trimming and truncating the dendrogram 

fig, ax = plt.subplots()
ax.set_title('Hierarchical Clustering Dendrogram (truncated)')
ax.set_xlabel('sample index')
ax.set_ylabel('distance')
dendrogram(
    Z,
    truncate_mode='lastp',  # show only the last p merged clusters
    p=6,
    show_leaf_counts=False, # otherwise numbers in brackets are counts
    leaf_rotation=90.,
    leaf_font_size=12.,
    show_contracted=True    # to get a distribution impression in
                            #truncated branches
);

## Hierarchical clustering with `sklearn` on Iris

**[Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html)** for AgglomerativeClustering in `sklearn`


**[A great example of using Manhattan distance](https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_clustering_metrics.html#sphx-glr-auto-examples-cluster-plot-agglomerative-clustering-metrics-py)** with agglomerative clustering in `sklearn`.

In [ ]:
# we can also use the scikitlearn module hierarchical clustering
# to perform the same task

np.random.seed(2000)

In [ ]:
# try clustering on the iris dataset

iris = load_iris()

# in this case, we won't be working with predicting labels,
# so we will only use the features (X)

X_iris = iris.data
y_iris = iris.target

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X_iris[:, 1], X_iris[:, 3], c=y_iris);

In [ ]:
# Comparing our clusters with truth. Note that we are fitting our
# model to ALL the columns but only plotting two of them!

iris_cluster = AgglomerativeClustering(n_clusters=3)
iris_cluster

In [ ]:
pred_iris_clust = iris_cluster.fit_predict(X_iris)
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].scatter(X_iris[:, 1], X_iris[:, 3],
             c=y_iris)
ax[1].scatter(X_iris[:, 1], X_iris[:, 3],
           c=pred_iris_clust);

In [ ]:
def switch(x):
    if x == 1:
        return 0
    elif x == 0:
        return 1
    else:
        return x

In [ ]:
pred_iris_clust = iris_cluster.fit_predict(X_iris)
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].scatter(X_iris[:, 1], X_iris[:, 3],
             c=y_iris)
ax[1].scatter(X_iris[:, 1], X_iris[:, 3],
           c=list(map(switch, pred_iris_clust)));

## Evaluate

To evaluate you might try different numbers of clusters and compare their silhouette score as you did with $k$-means.

In [ ]:
# evaluation: silhouette score

silhouette_score(X_iris, pred_iris_clust)

### Evaluating number of clusters / Cut points

For hierarchical agglomerative clustering, or clustering in general, it is generally difficult to truly evaluate the results. Therefore, it is up you, the data scientists, to decide.

**[Stanford has a good explanation on page 380](https://nlp.stanford.edu/IR-book/pdf/17hier.pdf)** of your options for picking the cut-off. 

When we are viewing dendrograms for hierarchical agglomerative clustering, we can visually examine where the natural cutoff is, despite it not sounding exactly statistical, or scientific. We might want to interpret the clusters and assign meanings to them depending on domain-specific knowledge and shape of dendrogram. However, we can evaluate the quality of our clusters using measurements such as Sihouette score discussed in the k-means lectures. 

# Advantages & Disadvantages of hierarchical clustering

## Advantages

- Intuitive and easy to implement
- More informative than k-means because it takes individual relationship into consideration
- Allows us to look at dendrogram and decide number of clusters

## Disadvantages

- Very sensitive to outliers
- Cannot undo the previous merge, which might lead to problems later on 

### Further reading

- [from MIT on just hierarchical](http://web.mit.edu/6.S097/www/resources/Hierarchical.pdf)
- [from MIT comparing clustering methods](http://www.mit.edu/~9.54/fall14/slides/Class13.pdf)
- [fun CMU slides on clustering](http://www.cs.cmu.edu/afs/andrew/course/15/381-f08/www/lectures/clustering.pdf)